# Problem Formation

Given a Pattern String as an input, we want to know if it contains dark pattern in it. We use a balanced dataset cotaining all the instances in the Princeton dataset which are all dark patterns, and the instances in the 'normie.csv' file which are labeled as NOT dark patterns. Hence we have a balanced dataset consisting of pattern strings with dark pattern and without park patterns.

Then we use this labeled dataset to build and train supervised machine learning models, and select most suitable ones for our project.

----


In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary.
from sklearn.feature_extraction.text import TfidfVectorizer

# Bernoulli Naive Bayes (Similar as  MultinomialNB), this classifier is suitable for discrete data. The difference between MultinomialNB and BernoulliNB is that while  MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolen features, which means in the case of text classification, word occurrence vectores (rather than word count vectors) may be more suitable to be used to train and use this classifier.
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

import matplotlib.pyplot as plt
# import seaborn as sns

## Data Exploration

---
Import the merged dataset, and explore the dataset.

In [2]:
data = pd.read_csv('enriched_confirm.csv')

In [3]:
data.head(5)

,Pattern String,classification
0,Ends in 07:42:09,Not_Dark
1,Ends in 07:37:10,Not_Dark
2,Ends in 02:27:10,Not_Dark
3,Ends in 04:17:10,Not_Dark
4,Ends in 01:57:10,Not_Dark


---
`check the dataset information`

There are 7952 NOT NULL instances of pattern strings in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8187 entries, 0 to 8186
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  8187 non-null   object
 1   classification  8187 non-null   object
dtypes: object(2)
memory usage: 128.0+ KB


In [5]:
# check the distribution of the target value --- classification.

print('Distribution of the tags:\n{}'.format(data['classification'].value_counts()))

Distribution of the tags:
Not_Dark    7994
Dark         193
Name: classification, dtype: int64


In [6]:
# Change the content into lowercase

data['Pattern String'] = data['Pattern String'].str.lower()

In [7]:
# For later training the model, we should remove the duplicate input to reduce overfitting.

data = data.drop_duplicates(subset="Pattern String")

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))


Distribution of the tags:
Not_Dark    7755
Dark         180
Name: classification, dtype: int64


---
## Data Preparation

In [8]:
# split the dataset into train and test dataset as a ratio of 80%/20% (train/test).

string_train, string_test, dark_train, dark_test = train_test_split(
    data['Pattern String'], data["classification"], train_size = .8)

---
`Encode the target vales into integers` --- 'classification'

In [9]:
encoder = LabelEncoder()
encoder.fit(dark_train)
y_train = encoder.transform(dark_train)
y_test = encoder.transform(dark_test)

In [10]:
# check the mapping of encoding results (from 0 to 1 representing 'Dark', 'Not Dark')

integer_mapping = {label: encoding for encoding, label in enumerate(encoder.classes_)}
print(integer_mapping)

{'Dark': 0, 'Not_Dark': 1}


In [11]:
# Check the frequency distribution of the training pattern classification with pattern classification names.

(unique, counts) = np.unique(dark_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['Dark' 133]
 ['Not_Dark' 6215]]


In [12]:
# Check the frequency distribution of the encoded training pattern classification with encoded integers.

(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  133]
 [   1 6215]]


In [13]:
# Check the frequency distribution of the encoded testing pattern classification with encoded integers.

(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0   47]
 [   1 1540]]


---
`Encode the textual features into series of vector of numbers`

In [14]:
# get the word count vector of the pattern string to encode the pattern string.

tv = TfidfVectorizer()
tv.fit(string_train)

x_train = tv.transform(string_train)
x_test = tv.transform(string_test)

x_train.shape, x_test.shape

((6348, 5594), (1587, 5594))

In [15]:
# save the CountVectorizer to disk

joblib.dump(tv, 'presence_TfidfVectorizer.joblib')

['presence_TfidfVectorizer.joblib']

---
# Rough Idea about the effect of different classifiers
---

In [16]:
# Four models are tested:
# -- Logistic Regression
# -- Linear Support Vector Machine
# -- Random Forest
# -- Bernoulli Naive Bayes

classifiers = [LogisticRegression(), LinearSVC(), RandomForestClassifier(), BernoulliNB()]

In [17]:
# Calculate the accuracies of different classifiers using default settings.

acc = []
pre = []
cm = []

for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    acc.append(metrics.accuracy_score(y_test, y_pred))
    pre.append(metrics.precision_score(y_test, y_pred, pos_label=0))
    cm.append(metrics.confusion_matrix(y_test, y_pred))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# List the accuracies of different classifiers.

for i in range(len(classifiers)):
    print("{} accuracy: {:.3f}".format(classifiers[i],acc[i]))
    print("{} precision: {:.3f}".format(classifiers[i],pre[i]))
    print("Confusion Matrix: {}".format(cm[i]))

LogisticRegression() accuracy: 0.988
LogisticRegression() precision: 0.967
Confusion Matrix: [[  29   18]
 [   1 1539]]
LinearSVC() accuracy: 0.992
LinearSVC() precision: 0.947
Confusion Matrix: [[  36   11]
 [   2 1538]]
RandomForestClassifier() accuracy: 0.992
RandomForestClassifier() precision: 0.972
Confusion Matrix: [[  35   12]
 [   1 1539]]
BernoulliNB() accuracy: 0.970
BernoulliNB() precision: 0.000
Confusion Matrix: [[   0   47]
 [   0 1540]]


---
# Bernoulli Naive Bayes Classifier


---
### `Use default setting of classifier hyperparameters`

In [19]:
clf_bnb = BernoulliNB().fit(x_train, y_train)

y_pred = clf_bnb.predict(x_test)

In [20]:
clf_bnb.get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

---
`use the default setting of hyperparameters of the Bernoulli Naive Bayes classifier`

In [21]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9703843730308759
Precision: 0.0
Confusion Matrix:
 [[   0   47]
 [   0 1540]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   1, 1587]])

---
### `Parameter Tunning of BernoulliNB classifier`
`Define the combination of parameters to be considered`

In [23]:
param_grid = {'alpha':[0,1], 
              'fit_prior':[True, False]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [24]:
gs = GridSearchCV(clf_bnb,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [25]:
best_bnb = gs.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    1.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.7s finished


In [26]:
scores_df = pd.DataFrame(best_bnb.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_alpha', 'param_fit_prior']]

,rank_test_score,mean_test_score,param_alpha,param_fit_prior
0,1,0.978576,1,True
1,2,0.978103,1,False
2,3,0.972118,0,True
3,4,0.963927,0,False


In [27]:
best_bnb.best_params_

{'alpha': 1, 'fit_prior': True}

In [28]:
y_pred_best = best_bnb.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   1 1587]]


In [29]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9703843730308759
Precision: 0.0
Confusion Matrix:
 [[   0   47]
 [   0 1540]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---
`Save the best BernoulliNB model for future use`

In [30]:
# save the model to local disk

joblib.dump(best_bnb, 'bnb_presence_classifier.joblib')

['bnb_presence_classifier.joblib']

---
# Random Forest Classifier


---
### `Use default setting of classifier hyperparameters`

In [31]:
clf_rf = RandomForestClassifier().fit(x_train, y_train)

y_pred = clf_rf.predict(x_test)

In [32]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [33]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9911783238815375
Precision: 0.9714285714285714
Confusion Matrix:
 [[  34   13]
 [   1 1539]]


In [34]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   35],
       [   1, 1552]])

---
### `Parameter Tunning of Random Forest classifier`
`Define the combination of parameters to be considered`

In [35]:
param_grid = {'bootstrap':[True,False], 
              'criterion':['gini','entropy'],
              'max_depth':[10,20,30,40,50, None],
              'min_samples_leaf':[1,2,4],
              'min_samples_split':[2,5,10],
              'n_estimators':[100,200,300]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [36]:
gs = GridSearchCV(clf_rf,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [37]:
best_rf = gs.fit(x_train,y_train)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 33.8min
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed: 35.8min finished


In [38]:
scores_df = pd.DataFrame(best_rf.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_bootstrap', 'param_criterion','param_max_depth','param_min_samples_leaf','param_min_samples_split','param_n_estimators']]

,rank_test_score,mean_test_score,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators
0,1,0.997795,False,entropy,None,2,2,100
1,2,0.997637,False,entropy,None,2,2,200
2,3,0.997637,False,gini,None,2,5,100
3,3,0.997637,False,entropy,None,2,10,300
4,5,0.997637,False,entropy,None,2,2,300
...,...,...,...,...,...,...,...,...
643,627,0.979049,True,entropy,10,2,10,100
644,627,0.979049,True,entropy,10,2,10,200
645,627,0.979049,True,gini,10,4,10,100
646,627,0.979049,False,gini,10,2,2,200


In [39]:
best_rf.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [40]:
y_pred_best = best_rf.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   35]
 [   1 1552]]


In [41]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9911783238815375
Precision: 0.9714285714285714
Confusion Matrix:
 [[  34   13]
 [   1 1539]]


---
`Save the best Random Forest model for future use`

In [42]:
# save the model to local disk

joblib.dump(best_rf, 'rf_presence_classifier.joblib')

['rf_presence_classifier.joblib']

---
# SVM Classifier


---
### `Use default setting of classifier hyperparameters`

In [43]:
clf_svm = LinearSVC().fit(x_train,y_train)

y_pred = clf_svm.predict(x_test)

In [44]:
clf_svm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [45]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9918084436042848
Precision: 0.9473684210526315
Confusion Matrix:
 [[  36   11]
 [   2 1538]]


In [46]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   38],
       [   1, 1549]])

---
### `Parameter Tunning of SVM classifier`
`Define the combination of parameters to be considered`

In [47]:
param_grid = {'C':[0.1,1,10,100],
              'penalty':['l1','l2']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [48]:
gs = GridSearchCV(clf_svm,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [49]:
best_svm = gs.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished


In [50]:
scores_df = pd.DataFrame(best_svm.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_C']]

,rank_test_score,mean_test_score,param_penalty,param_C
0,1,0.997007,l2,1
1,2,0.997007,l2,10
2,3,0.996534,l2,100
3,4,0.993541,l2,0.1
4,5,NaN,l1,0.1
5,6,NaN,l1,1
6,7,NaN,l1,10
7,8,NaN,l1,100


In [51]:
best_svm.best_params_

{'C': 1, 'penalty': 'l2'}

In [52]:
y_pred_best = best_svm.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   38]
 [   1 1549]]


In [53]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9918084436042848
Precision: 0.9473684210526315
Confusion Matrix:
 [[  36   11]
 [   2 1538]]


---
`Save the best SVM model for future use`

In [54]:
# save the model to local disk

joblib.dump(best_svm, 'svm_presence_classifier.joblib')

['svm_presence_classifier.joblib']

---
# Logistic Regression Classifier


---
### `Use default setting of classifier hyperparameters`

In [55]:
clf_lr = LogisticRegression().fit(x_train, y_train)

y_pred = clf_lr.predict(x_test)

In [56]:
clf_lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Logistic Regression classifier.`

In [57]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9880277252678009
Precision: 0.9666666666666667
Confusion Matrix:
 [[  29   18]
 [   1 1539]]


In [58]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   30],
       [   1, 1557]])

---
### `Parameter Tunning of Logistic Regression classifier`
`Define the combination of parameters to be considered`

In [59]:
param_grid = {'penalty':['l1','l2'], 
              'solver':['lbfgs','newton-cg','sag']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [60]:
gs = GridSearchCV(clf_lr,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [61]:
best_lr = gs.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s finished


In [62]:
scores_df = pd.DataFrame(best_lr.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_solver']]

,rank_test_score,mean_test_score,param_penalty,param_solver
0,1,0.990863,l2,lbfgs
1,1,0.990863,l2,newton-cg
2,1,0.990863,l2,sag
3,4,NaN,l1,lbfgs
4,5,NaN,l1,newton-cg
5,6,NaN,l1,sag


In [63]:
best_lr.best_params_

{'penalty': 'l2', 'solver': 'lbfgs'}

In [64]:
y_pred_best = best_lr.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   30]
 [   1 1557]]


In [65]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9880277252678009
Precision: 0.9666666666666667
Confusion Matrix:
 [[  29   18]
 [   1 1539]]


---
`Save the best Logistic Regression model for future use`

In [66]:
# save the model to local disk

joblib.dump(best_lr, 'lr_presence_classifier.joblib')

['lr_presence_classifier.joblib']